# Aztec G-dDT Test Run

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import os
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = "png"

## Perform Simulation

In [2]:
from aztec_gddt.experiment import test_run
from aztec_gddt.scenario_experiments import scenario_group_1_experiment


exp = scenario_group_1_experiment()


from cadCAD.engine import ExecutionContext, ExecutionMode, Executor


_exec_mode = ExecutionMode().single_mode
exec_context = ExecutionContext(_exec_mode, additional_objs={'deepcopy_off': True})
executor = Executor(exec_context=exec_context, configs=exp.configs, supress_print=False)

# Execute the cadCAD experiment
(records, tensor_field, _) = executor.execute()

# Parse the output as a pandas DataFrame
df = pd.DataFrame(records)

# Drop substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = first_ind | last_ind
df = df.loc[inds_to_drop].drop(columns=['substep'])


# Assign Params
M_dict = exp.configs[0].sim_config['M']
params_set = set(M_dict.keys())

selected_params = params_set
# Attribute parameters to each row*

from cadCAD.tools.execution.easy_run import select_config_M_dict # type: ignore
params_dict = select_config_M_dict(exp.configs, 0, selected_params)

# Handles all cases of parameter types including list
for key, value in params_dict.items():
    df[key] = df.apply(lambda _: value, axis=1)

for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    params_dict = select_config_M_dict(exp.configs, i, selected_params)
    for key, value in params_dict.items():
        df.loc[n_df.index, key] = df.loc[n_df.index].apply(
            lambda _: value, axis=1)
        

sim_df = df


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.5.3
Execution Mode: single_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (2, 2000, 78, 30, 32)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (1000, 39, 15, 32)
     Simulation 1: (Timesteps, Params, Runs, Sub-States) = (1000, 39, 15, 32)


Initializing configurations:   0%|          | 0/30 [00:00<?, ?it/s]

Execution Method: single_proc_exec


Flattening results:   0%|          | 0/180030 [00:00<?, ?it/s]

Total execution time: 9.79s


## Visualization over Model Features

In [3]:
sim_df.head(5)

,l1_blocks_passed,delta_l1_blocks,l2_blocks_passed,agents,validator_set,PROVING_COST_MODIFIER,current_epoch,last_epoch,last_reward_time_in_l1,last_reward,...,BLOCK_REWARD_SPEED_ADJ,MIN_STAKE,VALIDATOR_COMMITTEE_SIZE,SIGNATURES_NEEDED,BOND_SIZE,BOND_SLASH_PERCENT,AVERAGE_TX_COUNT_PER_SLOT,PROVING_COST_MODIFICATION_E,FEE_JUICE_PRICE_MODIFICATION_E,ORACLE_UPDATE_FREQUENCY_E
0,0,0,0,"[Agent(uuid='0', commitment_bond=1.28818418154...",{},0.0,"Epoch(init_time_in_l1=-999, validators=[], slo...","Epoch(init_time_in_l1=-999, validators=[], slo...",0,15000.0,...,0.00055,32,128,0.5,1,1.0,360,0.0,0.0,0.9
6,1,1,1,"[Agent(uuid='0', commitment_bond=1.28818418154...",{},0.0,"Epoch(init_time_in_l1=1, validators=['198', '2...","Epoch(init_time_in_l1=-999, validators=[], slo...",0,15000.0,...,0.00055,32,128,0.5,1,1.0,360,0.0,0.0,0.9
12,2,1,1,"[Agent(uuid='0', commitment_bond=1.28818418154...",{},0.0,"Epoch(init_time_in_l1=1, validators=['198', '2...","Epoch(init_time_in_l1=-999, validators=[], slo...",0,15000.0,...,0.00055,32,128,0.5,1,1.0,360,0.0,0.0,0.9
18,3,1,1,"[Agent(uuid='0', commitment_bond=1.28818418154...",{},0.0,"Epoch(init_time_in_l1=1, validators=['198', '2...","Epoch(init_time_in_l1=-999, validators=[], slo...",0,15000.0,...,0.00055,32,128,0.5,1,1.0,360,0.0,0.0,0.9
24,4,1,2,"[Agent(uuid='0', commitment_bond=1.28818418154...",{},0.0,"Epoch(init_time_in_l1=1, validators=['198', '2...","Epoch(init_time_in_l1=-999, validators=[], slo...",0,15000.0,...,0.00055,32,128,0.5,1,1.0,360,0.0,0.0,0.9


In [ ]:
fig = px.scatter(sim_df.reset_index(), x='l1_blocks_passed', y='l2_blocks_passed', color='subset', facet_col='run', facet_row='simulation')
fig.update_layout(yaxis_range=[0, 10], xaxis_range=[0,  10 * 3], width=1000, height=500)
fig.show()

In [ ]:
y = sim_df.current_epoch.map(lambda x: len(x.slots))

fig = px.line(sim_df, x='l1_blocks_passed', y=y, labels={"x": 'timestep', 'y': 'Slot Count for current epoch'}, color='subset', facet_col='run', facet_row='simulation')
fig.show()

In [ ]:
y = sim_df.last_epoch.map(lambda x: len(x.prover_quotes))

fig = px.scatter(sim_df, x='l1_blocks_passed', y=y, labels={"x": 'timestep', 'y': 'Prover Quotes'}, color='subset', facet_col='run', facet_row='simulation')
fig.show()

In [ ]:
y = sim_df.last_epoch.map(lambda x: x.accepted_prover_quote)

fig = px.line(sim_df, x='l1_blocks_passed', y=y, labels={"x": 'timestep', 'y': 'Accepted Prover Quote Value'}, color='subset', facet_col='run', facet_row='simulation')
fig.show()

## Visualization over PSUU Per-Trajectory Metrics

In [17]:
from aztec_gddt.psuu.metrics import *

In [ ]:
for label, fn in PER_TRAJECTORY_METRICS.items():
    print(f"{label}: {fn.__name__}")
    print(sim_df.reset_index().groupby(['simulation', 'subset', 'run']).apply(fn, include_groups=False))
    print("---")

In [ ]:
for label, fn in PER_TRAJECTORY_GROUP_METRICS.items():

    groups = list(sim_df.reset_index().groupby(['simulation']))
    for i, g in groups:
        dfs = [el[1] for el in list(g.groupby('run'))]
        print(f"{label}: {fn.__name__}")
        print(fn(dfs))
        print("---")    

In [4]:
control_params = [
    'RELATIVE_TARGET_MANA_PER_BLOCK',
    'MAXIMUM_MANA_PER_BLOCK',
    'MINIMUM_MULTIPLIER_CONGESTION',
    'UPDATE_FRACTION_CONGESTION',
    'OVERHEAD_MANA_PER_TX']

In [31]:
from aztec_gddt.psuu.metrics import *


for control_param in control_params:
    print(f"### {control_param} ###")
    groups = list(sim_df.reset_index().groupby([control_param]))

    for i, g in groups:
        print(f"{control_param} = {i}")
        for label, fn in PER_TRAJECTORY_GROUP_METRICS.items():
            dfs = [el[1] for el in list(g.groupby(['simulation', 'subset', 'run']))]
            print(f"{label}={fn(dfs) :.3f}")
        print("---")

### RELATIVE_TARGET_MANA_PER_BLOCK ###
RELATIVE_TARGET_MANA_PER_BLOCK = (0.1,)
TG-M1=0.342
TG-M2=13.300
TG-M3=1.300
TG-M4=0.983
TG-M5=nan
TG-M6=0.000
TG-M7=0.500
TG-M8=nan
TG-M9=nan
TG-M10=nan
TG-M11=nan
TG-M12=nan
TG-M13=nan
---
RELATIVE_TARGET_MANA_PER_BLOCK = (0.5,)
TG-M1=0.369
TG-M2=9.833
TG-M3=1.167
TG-M4=0.991
TG-M5=nan
TG-M6=0.000
TG-M7=0.417
TG-M8=nan
TG-M9=nan
TG-M10=nan
TG-M11=nan
TG-M12=nan
TG-M13=nan
---
RELATIVE_TARGET_MANA_PER_BLOCK = (0.9,)
TG-M1=0.431
TG-M2=12.000
TG-M3=1.250
TG-M4=0.980
TG-M5=nan
TG-M6=0.000
TG-M7=0.750
TG-M8=nan
TG-M9=nan
TG-M10=nan
TG-M11=nan
TG-M12=nan
TG-M13=nan
---
### MAXIMUM_MANA_PER_BLOCK ###
MAXIMUM_MANA_PER_BLOCK = (20000000,)
TG-M1=0.357
TG-M2=11.071
TG-M3=1.214
TG-M4=0.986
TG-M5=nan
TG-M6=0.000
TG-M7=0.643
TG-M8=nan
TG-M9=nan
TG-M10=nan
TG-M11=nan
TG-M12=nan
TG-M13=nan
---
MAXIMUM_MANA_PER_BLOCK = (40000000,)
TG-M1=0.394
TG-M2=12.000
TG-M3=1.250
TG-M4=0.985
TG-M5=nan
TG-M6=0.000
TG-M7=0.438
TG-M8=nan
TG-M9=nan
TG-M10=nan
TG-M11=nan
TG-M12=n